## 3 - Neural Machine Translation by Jointly Learning to Align and Translate

> Ở notebook thứ 3 này, chúng ta sẽ cùng implementing model từ paper [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/pdf/1409.0473.pdf). Model này đạt được perplexity xấp xỉ 27, so với 34 từ các model trước.

### Introduction

> Đây là mô hình encoder-decoder được sử dụng từ các notebooks:

> ![figure1](./images/3.seq2seq.png)

> Ở model trước, chúng ta cùng thiết lập một kiến trúc để giảm việc nén thông tin bằng cách truyền context vector z vào decoder ở mỗi thời điểm. Và, ta truyền cả context vector và embedded input word, $\bold{d(y_t)}$ cùng với hidden state $s_t$ vào linear layer, f để đưa ra dự đoán.

![figure2](./images/3.seq2seq_2.png)

> Mặc dù giảm được việc nén thông tin, context vector vẫn phải cần lưu trữ thông tin về source sentence.

> Trong notebook này, chúng ta sẽ xây dựng một model xóa bỏ việc đè nén thông tin bằng cách cho phép decoder quan sát toàn bộ source sentence (thông qua các hidden states của nó) ở mỗi bước decoding. Và, chúng ta sẽ sử dụng *attention*. Chi tiết về *attention* sẽ được trình bày ở mục sau.

### Preparing Data

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

ModuleNotFoundError: No module named 'torchtext.legacy'

> Thiết lập random seeds.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Load model tokenize tiếng Anh và tiếng Đức của spaCy.

In [ ]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

> Tạo tokenizers.

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [2]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '', 
            eos_token = '', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '', 
            eos_token = '', 
            lower = True)

NameError: name 'Field' is not defined

> Load data.

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

> Build vocabulary.

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

### Building the Seq2Seq Model

##### Encoder

> Đầu tiên, ta sẽ xây dựng encoder. Tương tự như model ở notebook trước, chúng ta chỉ sư dụng một layer GRU, tuy nhiên bây giờ ta sẽ sử dụng *bidirectional RNN*. Với *bidirectional RNN*, ta có 2 lớp RNNs ở mỗi layer. Quá trình foward của layer RNN đi từ trái sang phải (green), quá trình backward đi từ phải sang trái (teal). Trong pytorch, ta chỉ cần set `bidirectional = True` để sử dụng *bidirectional RNN*.

![figure3](./images/3.seq2seq_bidirectional.png)

> Giờ đây, chúng ta có: <br> <br>
> $\begin{aligned} h_t^{\rightarrow} &=\text { EncoderGRU } \rightarrow\left(e\left(x_t^{\rightarrow}\right), h_{t-1}^{\rightarrow}\right) \\ h_t^{\leftarrow} &=\text { EncoderGRU } \end{aligned}$ <br> <br>
> Với $x_0^{\rightarrow}$ = \<sos>, $x_1^{\rightarrow}$ = gutten và $x_0^{\rightarrow}$ = \<eos>, $x_1^{\rightarrow}$ = morgen.

> Trước đây, chúng ta chỉ truyền 1 đầu vào (`embedded`) vào RNN, và thông báo với Pytorch rằng hãy khởi tạo forward và backward initial hidden states ($h_0^{\rightarrow}$ và $h_0^{\leftarrow}$) bằng tensor 0. Sau quá trình xử lý của RNN, ta thu được 2 context vectors, 1 là từ quá trình forward, $z^{\rightarrow}$ = $h_T^{\rightarrow}$ và 1 từ quá trình backward, $z^{\leftarrow}$ = $h_T^{\leftarrow}$.

> RNN trả về `outputs` và `hidden`.

> `outputs` có kích thước **[src len, batch size, hid dim * num directions]**. Phần tử `hid_dim` đầu tiên ở trục thứ 3 là hidden states từ top layer forward RNN, và phần tử `hid_dim` cuối cùng là từ top layer RNN backward. Ta có thể hiểu trục thứ 3 gồm forward và backward hidden states được ghép nối với nhau: $h_1$ = [$h_1^{\rightarrow}$;$h_T^{\leftarrow}$], $h_2$ = [$h_2^{\rightarrow}$;$h_{T-1}^{\leftarrow}$], ... và, ta ký hiệu tất cả các encoder hidden states (gồm hidden states từ forward và backward được ghép nối với nhau) là H = {$h_1, h_2, h_3, ..., h_T$}

> `hidden` có kích thước **[n layers * num directions, batch size, hid dim]**. Sử dụng **[-2,:,:]** cho ta hidden state của top layer forward RNN sau bước cuối cùng và **[-1,:,:]** cho ta hidden state của top layer backward RNN sau bước cuối cùng.

> Decoder là undirectional, nó chỉ cần một context vector z đóng vai trò initial hidden state $s_0$. Mà hiện tại ta đang có 2 context vector ($z^{\rightarrow}$ = $h_T^{\rightarrow}$ và $z^{\leftarrow}$ = $h_T^{\leftarrow}$). Đơn giản, ta chỉ cần ghép nối 2 vector này lại và cho đi qua một lớp fully connected để giảm về chiều của một vector (ở đây, activation function ta chọn là hàm tanh): <br> 
>> z = tanh(g($h_T^{\rightarrow}$, $h_T^{\leftarrow}$)) = tanh(g($z^{\rightarrow}$, $z^{\leftarrow}$)) = $s_0$.

> **NOTE**: Trong paper, người ta chỉ lấy hidden states từ backward RNN và đưa vào linear layer để thu context vector và decoder initial hidden state.

> Do model chúng ta cần quan sát toàn bộ source sentence nên chúng ta sẽ return `outputs`, stacked forward và backward hidden states của mỗi token trong source sentence. Ngoài ra, ta cần return `hidden` để làm initial hidden state ở decoder.

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                input_dim,
                emb_dim,
                hid_dim,
                n_layers,
                dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)

        if n_layers > 1:
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout = dropout, bidirectional = True)
        else:
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, bidirectional = True)

        self.fc = nn.Linear(hid_dim * 2, hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src len, batch size, emb dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]

        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer

        # hidden [-2, :, : ] is the last of the forwards RNN
        # hidden [-1, :, : ] is the last of the backwards RNN
        # initial decoder hidden is final hidden state of the forwards and backwards
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [batch size, hid dim]
        return outputs, hidden

##### Attention

> Tiếp theo là attention layer. Layer này sẽ nhận đầu vào là previous hidden state của decoder $s_{t-1}$ và tất cả cách stacked foward và backward hidden states từ encoder, **H**. Layer sẽ trả về một attention vector $a_t$ có kích thước là chiều dài của source sentence, mỗi phần tử của vector nằm trong khoảng (0,1) và tổng các phần từ bằng 1.

> Bản chất là layer này sẽ lấy đối tượng đã được decode tính tới thời điểm hiện tại $s_{t-1}$ và tất cả những gì chúng ta đã encode, **H** để sinh ra một attention vector $a_t$, vector này biểu diễn việc model nên chú ý tới từ nào trong source sentence để đưa ra dự đoán chính xác cho từ tiếp theo $\hat{y}_{y+1}$.

> Đầu tiên, ta tính toán *energy* giữa previous decoder hidden state và encoder hidden states. 